In [1]:
#2nd Day of expt: flash both numbers at correct opacity, asks subjects the first 3 numbers they say following a cue
#Possible modifications: Ask for just 1 number

#What did we learn?
#   -To fix problem: find out in small steps what effects what
#   -Need to update window color before draw textBox!
#   -Can put a "break" to get out of the immediate LOOP (not if statement --> can conditionalize it) to only go through it once(or so many times)
#Actual Expt, records subject perception of color change
#Qs: Check: Leaving Expt A/B with cues as w/b not just space since using them for timing and things
#ToDo: Set up for 200 total trials (expect to throw out ~100)
#      Make clear subject should take their time when inputting the numbers --> Maybe put in explaining part
#Notes: Subject error is expected, experiemnt does not code to help them, mess up trials are lost.
#       In Synchonous condiditon, expect a ~300 ms delay in response (Sperling). In asynchonous condiiton, expect this delay +
#       additional delay we're hypothesizing
#       Will calculate number timing after the fact since have the data to do it
#       Trials where color was dominant when cued (synchonous) are like control trials. They are not what we're testing tho, keep sep
#       B/W asynchronous trials will be averaged together (no real difference)

from psychopy import visual, event, core
from psychopy.hardware import keyboard
import numpy as np

win = visual.Window(fullscr=True, monitor="testMonitor", units="pix", color=(0,0,0), colorSpace='rgb')
keyb = keyboard.Keyboard()

#Presentation variables
numberSpace = 256
whiteSize = 270
whiteDot = 4
disp = 300
texMod = 462
texRing = np.random.normal(0, 0.5, (whiteSize, whiteSize))
ring = np.ones((numberSpace, numberSpace)) #Really just a uniform color ring
whiteFix = np.ones((whiteDot, whiteDot))
duration = 480                    #length of each trial, p sure refreshing at 60Hz, 8s trials atm
opac = 1.0

#Trial variables
#For expter to change
subjNum = 0 #Subject number
trialNum = 4 #Total number of trials. go fro 200 total trials in act expt 20 per block, each of 4 options runs 3x randomly in a trial in actual expt
blockNum = 1 #Blocks is 4 atm to get 12 trials in a block
freqWhite = 7.5 #f (Hz) want the white numbers flashing
freqBlack = 10 #f (Hz) want the black numbers flashing
refRate = 60 #Refresh rate (Hz) of computer monitor
#Fixed
trialCounter = 0 #Keeps track of current trial number
options = [] #Each possible trial type coded. 0 is WL cue W, 1 = WL cue B, 2 = BL cue B, 3 is BL cue W
data = np.zeros([trialNum,5], dtype=object)
frameSwitchWhite = int(refRate/freqWhite)
frameSwitchBlack = int(refRate/freqBlack)

#White Num Stim
texStimR = visual.GratingStim(win=win,tex=texRing,mask="circle",size=(whiteSize, whiteSize),autoDraw=True,pos=[-disp,0])
ringStimR = visual.GratingStim(win=win,tex=ring,mask="circle",size=(numberSpace, numberSpace),autoDraw=True,pos=[-disp,0],color=(0,0,0))
whiteNumL = visual.TextStim(win,bold=True,units='pix',pos=[-disp+texMod,0],text=str(np.random.randint(0,10)), color=(1.0,1.0,1.0), opacity=opac, height=90.0)
whiteNumR = visual.TextStim(win,bold=True,units='pix',pos=[disp+texMod,0],text=str(np.random.randint(0,10)), color=(1.0,1.0,1.0), opacity=opac, height=90.0)
fixationR = visual.GratingStim(win=win,tex=whiteFix,mask="circle",size=(whiteDot, whiteDot),autoDraw=True,pos=[-disp,0])

#Black Num Stim
texStimL = visual.GratingStim(win=win,tex=texRing,mask="circle",size=(whiteSize, whiteSize),autoDraw=True,pos=[disp,0])
ringStimL = visual.GratingStim(win=win,tex=ring,mask="circle",size=(numberSpace, numberSpace),autoDraw=True,pos=[disp,0],color=(0,0,0))
blackNumR = visual.TextStim(win,bold=True,units='pix',pos=[disp+texMod,0],text=str(np.random.randint(0,10)), color=(-1.0,-1.0,-1.0), opacity=opac, height=90.0)
blackNumL = visual.TextStim(win,bold=True,units='pix',pos=[-disp+texMod,0],text=str(np.random.randint(0,10)), color=(-1.0,-1.0,-1.0), opacity=opac, height=90.0)
fixationB = visual.GratingStim(win=win,tex=whiteFix,mask="circle",size=(whiteDot, whiteDot),autoDraw=True,pos=[disp,0])

#Input Number Stim
textbox = visual.TextBox(window=win,text='Please',font_size=15,font_color=[-1,-1,-1],background_color=[1,1,1],border_color=[-1,-1,-1,1],
                         textgrid_shape=[34,2],pos=(0.0,.6),)
myPromptB = "input the first 3 BLACK numbers you saw: "
myPromptW = "input the first 3 WHITE numbers you saw: "
inputText = ""



#Run Experiement
for j in range(blockNum):
    for i in range(int(trialNum/blockNum/4)):
        options.extend([0,1,2,3])
    for i in range(int(trialNum/blockNum)): #should go through 4 times in testing run, hit each option once
        #Initialize
        RT = 0
        whiteList = [] #presented white numbers
        blackList = [] #presented black numbers
        t = np.random.choice(options)
        ringChangeTime = np.random.randint(120,300)  #will happen between 2 and 5 seconds
        options.remove(t)
        #Instructions and turn on numbers
        instr = visual.TextStim(win,bold=True,units='pix',pos=[-150+texMod,250],text=str("Align Fixations, Press Space"), color=(1.0,1.0,1.0), opacity=1.0, height=20.0)
        instr.autoDraw = True
        if (t==0 or t==1): #WL   
            whiteNumL.autoDraw=True
            blackNumR.autoDraw=True
        if (t==2 or t==3): #WR   
            whiteNumR.autoDraw=True
            blackNumL.autoDraw=True
        win.flip()
        core.wait(1) #Prevents subject from hitting key on previous trial to start next one
        event.waitKeys()
        instr.autoDraw = False
        #Run Trial
        keyb.clock.reset()  # reset keyboard timer
        for i in range(duration):
            keys = keyb.getKeys(['space'], waitRelease=False)
            if (i%frameSwitchWhite == 0):
                whiteList.append(whiteNumL.text)
                if (t==0 or t==1):
                    whiteNumL.text = str(np.random.randint(0,10))
                    whiteList.append(whiteNumL.text)
                if (t==2 or t==3):
                    whiteNumR.text = str(np.random.randint(0,10))
                    whiteList.append(whiteNumR.text)
            if ((i == ringChangeTime) and (t==0 or t==3)):  #Cuing White
                win.color=(1,1,1)
                keyb.clock.reset() # reset keyboard timer
            if ((i == ringChangeTime) and (t==1 or t==2)): #Cuing black
                win.color=(-1,-1,-1)
                keyb.clock.reset() # reset keyboard timer
            if (i%frameSwitchBlack == 0):    
                blackList.append(blackNumR.text)
                if (t==0 or t==1):
                    blackNumR.text = str(np.random.randint(0,10))
                    blackList.append(blackNumR.text)
                if (t==2 or t==3):
                    blackNumL.text = str(np.random.randint(0,10))
                    blackList.append(blackNumL.text)
            for key in keys:
                if key == 'space':
                    RT += key.rt #Measures keypress onset (I think)
            win.flip()

        #Get subj input on numbers seen
        a = 0
        win.color=(0,0,0)
        win.flip()
        if (t==0 or t==3): #Cue was White, puts up before numbers are input     
                textbox.setText('Please '+myPromptW + inputText)
        if (t==1 or t==2): #Cue was Black
                textbox.setText('Please '+myPromptB + inputText)
        textbox.draw()
        win.flip()
        while (a<3):
            core.wait(0.1)
            keys = keyb.getKeys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                               'num_1', 'num_2', 'num_3','num_4', 'num_5', 'num_6','num_7', 'num_8', 'num_9', 'num_0'], waitRelease=True)
            for key in keys:
                num = key.name[-1]
                inputText = inputText + num
                if (t==0 or t==3): #Cue was White     
                    textbox.setText('Please '+myPromptW + inputText)
                if (t==1 or t==2): #Cue was Black
                    textbox.setText('Please '+myPromptB + inputText)
                a+=1
                win.color=(0,0,0)
                textbox.draw()
                win.flip()
        core.wait(1)
        if (t==0 or t==3):
            results = [subjNum,t,RT,whiteList,inputText]
            data[trialCounter] = results #Inputs trial data in corresponding row
        if (t==1 or t==2):
            results = np.array([subjNum,t,RT,blackList,inputText])
            data[trialCounter] = results
        trialCounter+=1
        
        #Turn off all numbers
        whiteNumL.autoDraw=False
        blackNumR.autoDraw=False
        whiteNumR.autoDraw=False
        blackNumL.autoDraw=False
        win.color=(0,0,0)
        inputText = ""
        textbox.setText('Please') #resets textbox text with no seen user input text
        win.flip()
        
    #Give Subjects a Break
    if (j < blockNum-1):
        instr = visual.TextStim(win,bold=True,units='pix',pos=[-250+texMod,250],text=str("Take a Break! Let Experimentor know."), color=(1.0,1.0,1.0), opacity=1.0, height=20.0)
        instr.autoDraw = True
        win.flip()
        event.waitKeys()
        event.waitKeys()
        instr.autoDraw = False

#Tell Subjects they're done
instr = visual.TextStim(win,bold=True,units='pix',pos=[-250+texMod,250],text=str("You're all done! Let Experimentor know."), color=(1.0,1.0,1.0), opacity=1.0, height=20.0)
instr.autoDraw = True
win.flip()
event.waitKeys()
event.waitKeys()
instr.autoDraw = False

#Close Window and Output Data
win.close()
print(data)

UnboundLocalError: local variable 'trialCounter' referenced before assignment

In [ ]:
numbers = input("What were the first 3 numbers you saw? ")


In [ ]:
#
from psychopy.hardware import keyboard
from psychopy import core, event

kb = keyboard.Keyboard()

# during your trial
kb.clock.reset()  # when you want to start the timer from

a = 0
while (a<3):
    print('a')
    core.wait(0.5)
    keys = kb.getKeys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','w','x','y'], waitRelease=True)
    for key in keys:
        print(key.name)
        a+=1

In [5]:
import numpy as np

trialNum = 4
data = np.zeros([trialNum,5])
trialCounter = 0

results = [0,1,2,3,4]
data[trialCounter,:] = results #Inputs trial data in corresponding row
print(data)

[[0. 1. 2. 3. 4.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [6]:
i = "ft"
def func():
    print(i+"5")
func()

ft5
